In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install -q datasets hydra-core evaluate trl accelerate bitsandbytes peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.8/410.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 67.0 M

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Hydra

In [ ]:
import os

import hydra
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from hydra.core.global_hydra import GlobalHydra
from hydra.core.config_store import ConfigStore
from omegaconf import OmegaConf

In [ ]:
with initialize_config_dir(version_base=None,
                           config_dir="/content/drive/MyDrive/hydra_configs/fine_tune",
                           job_name="fine_tune"):
    config = compose(config_name='config.yaml')

# Mistral fine-tune

## Load dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("HuggingFaceH4/no_robots")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.61k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/10.5M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/571k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
def format_for_causal_lm(example):
    """
    Converts a dictionary-based conversation example to a single string for CLM.
    """
    formatted_string = ""
    for message in example['messages']:
        if message.get('system') == 'system':
            formatted_string += f"<SYSTEM>: {message['content']}\n"
        elif message.get('role') == 'user':
            formatted_string += f"<USER>: {message['content']}\n"
        elif message.get('role') == 'assistant':
            formatted_string += f"<ASSISTANT>: {message['content']}\n"
    return {'text': formatted_string.strip()}

In [ ]:
from datasets import load_dataset

dataset = dataset.map(format_for_causal_lm)

Map:   0%|          | 0/9500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
dataset['train'] = dataset['train'].remove_columns(['prompt', 'prompt_id', 'messages', 'category'])

In [ ]:
print(len(dataset['train']))

9500


## Load model

In [ ]:
from huggingface_hub import login
login(token=config.hf.token)

In [ ]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 19.1 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch


tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", token=config.hf.token)
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", token=config.hf.token)
model.gradient_checkpointing_enable()

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

## Setup PEFT

In [ ]:
import pprint

pprint.pp(dict(config.lora_config))

{'r': 32,
 'lora_alpha': 64,
 'target_modules': ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'lm_head'],
 'bias': 'none',
 'lora_dropout': 0.05,
 'task_type': 'CAUSAL_LM'}


In [ ]:
!sudo add-apt-repository ppa:ubuntu-toolchain-r/test
!sudo apt-get update
!sudo apt-get install --only-upgrade libstdc++6

PPA publishes dbgsym, you may need to include 'main/debug' component
Repository: 'deb https://ppa.launchpadcontent.net/ubuntu-toolchain-r/test/ubuntu/ jammy main'
Description:
Toolchain test builds; see https://wiki.ubuntu.com/ToolChain

More info: https://launchpad.net/~ubuntu-toolchain-r/+archive/ubuntu/test
Adding repository.
Press [ENTER] to continue or Ctrl-c to cancel.
Adding deb entry to /etc/apt/sources.list.d/ubuntu-toolchain-r-ubuntu-test-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/ubuntu-toolchain-r-ubuntu-test-jammy.list
Adding key to /etc/apt/trusted.gpg.d/ubuntu-toolchain-r-ubuntu-test.gpg with fingerprint C8EC952E2A0E1FBDC5090F6A2C277A0A352154E5
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.i

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(**config.lora_config)

model = get_peft_model(model, lora_config)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


## Run finetuning

In [ ]:
pprint.pp(dict(config.train_params))

{'output_dir': 'Mistral-7B-Instruct-v0.3-norobots',
 'max_seq_length': 768,
 'num_train_epochs': 1,
 'per_device_train_batch_size': 8,
 'learning_rate': 1e-05,
 'optim': 'sgd'}


In [ ]:
from trl import SFTTrainer, SFTConfig

training_args = SFTConfig(**config.train_params)
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset.shuffle()['train'].select(range(2000))
)

trainer.train()

Converting train dataset to ChatML:   0%|          | 0/2000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

## Merge and save model

In [ ]:
adapter_model = trainer.model
merged_model = adapter_model.merge_and_unload()

trained_tokenizer = trainer.tokenizer

In [ ]:
merged_model.push_to_hub(config.hf.target_model_id, token=config.hf.token)
trained_tokenizer.push_to_hub(config.hf.target_model_id, token=config.hf.token)